Importer det vi trenger for geografiske analyser

In [ ]:
import pandas as pd
import geopandas as gpd
from shapely import wkt

Det vi trenger for å lage kart

In [ ]:
import hvplot.pandas
from holoviews import opts
import cartopy.crs as ccrs

# støtte for scroll med mus
opts.defaults( 
    opts.Overlay(
        width=900, height=300,
        active_tools=['wheel_zoom']
))

# Last inn bomstasjoner

Hint: 
  - ls lister opp filer. 
  - Først dataframe
    - Pass på tegnsett (encoding='utf-8') og semikolon (sep=';')
  - så dataframe['geometry'] =  dataframe['Kolonne med WKT geometri'].apply(wkt.loads)
  - til sist lager du geodataframe

*NB* For bruk i hvplot må geometrikolonnen hete "geometry"


In [ ]:
tmp = pd.read_csv( '45_bomstasjon-eksport.csv', sep=';', encoding='latin1')

tmp.head()

In [ ]:
tmp[['Geometri, punkt', 'geometri']].head(15)

# Bruk _Geometri, linje_ der den finnes

Vi ønsker å hente koordinater fra egenskapen _Geometri, punkt_ der den finnes. Hvis den mangler så henter vi plassering fra _geometri_ (avledet fra plassering i nettverket). 

### Metode 1: Sett verdi basert på betingelser (For vanskelig)

Les https://datatofish.com/if-condition-in-pandas-dataframe/ for tips om hvordan du setter "subslice" til gitt verdi basert på gitte betingelser. Har funnet mange oppskrifter hvor lambda-funksjonen spiser data fra én (og bare én) kolonne, for så å gjøre ulike ting basert på denne verdien. 


Mye lærerikt, men det ene tilfellet vi trenger: **Hent fra én kolonne hvis den finnes, hvis ikke henter du fra en annen** viser seg å bli overraskende komplest. 

### Metode 2: Iterer over hver rad 
```
# tmp = pd.read_csv( '45_bomstasjon-eksport.csv', sep=';', encoding='latin1') # Har lest fra før
tmp['geometry'] = ''

for index_label, row in tmp.iterrows(): 
    
    minshapelygeom = wkt.loads( row['geometri'] ) 
    if isinstance( row['Geometri, punkt'], str ): 
        minshapelygeom = wkt.loads( row['Geometri, punkt'])
    
    tmp.at[index_label, 'geometry'] = minshapelygeom
```
Fungerer helt greit, og er ikke for vanskelig

### Metode 3: Bruk fillna() for å hente de dataverdiene som mangler fra  kolonnen der vi har data

Se under


In [ ]:
# tmp = pd.read_csv( '45_bomstasjon-eksport.csv', sep=';', encoding='latin1') # Har lest fra før
tmp['Geometri, punkt'].fillna( tmp['geometri'], inplace=True)
tmp['geometry'] =  tmp['Geometri, punkt'].apply(wkt.loads)
bomstasjon = gpd.GeoDataFrame( tmp, geometry = 'geometry')
bomstasjon.crs = { 'init' : 'epsg:25833'}

In [ ]:
tmp[['Geometri, punkt', 'geometri']].head(15)

In [ ]:
bomstasjon['Navn bomstasjon'].fillna( '')
# ccrs.UTM(zone=33)
bomstasjon.hvplot( crs=ccrs.UTM(zone=33), hover_cols=['vegobjektid', 'Navn bomstasjon'
           'Takst liten bil', 'Takst stor bil' 'vegreferanse'], tiles='OSM',
            c='Takst liten bil', size=100, frame_height=600 )